# Double ML - modeling

`df_mix`

## 0. setup

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.preprocessing import LabelEncoder
import sklearn
import os
from matplotlib.pyplot import hist
import scipy.stats as stats
import math
import statsmodels.api as sm

In [2]:
# set random seed for numpy
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

In [3]:
def find_p(estimate, std):
    z_value = estimate / std
    p_value = stats.norm.sf(abs(z_value))*2
    return round(estimate, 4), round(std, 4), round(p_value, 4)

In [4]:
def label_encode_column(df, column):
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    return df

## 1. functions

### 1.1 Specify Nuisance Function Models

The next step is to specify models for 

*   $\mu(z,x)=\mathbb{E}(Y|z,x)$
*   $m(z,x) = P(A=1|z,x)$
*   $p(x) = P(Z=1|x)$

In [5]:
# make a function that returns a sklearn model for later use in k-folding
def make_mu_model():
  #return KNeighborsClassifier(n_neighbors=300)
  return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=300, max_depth=None)
  #return RandomForestClassifier(n_estimators=100, max_depth=5)

# specify a model for m(z,x)
def make_m_model():
  #return LogisticRegression(max_iter=1000, warm_start=True, random_state=RANDOM_SEED)
  #return RandomForestClassifier(n_estimators=200, max_depth=None)
  return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=200, max_depth=None)


#def make_p_model():
  #return RandomForestClassifier(n_estimators=200, max_depth=None) ###
  #return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=300, max_depth=None)

### 1.2 Functions that use cross fitting to get predicted $\hat{\mu}$, $\hat{m}$, $\hat{p}$ for each unit

In [6]:
# helper functions to implement the cross fitting

def m_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int, output_type:str):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    """
    predictions = np.full_like(A, np.nan, dtype=float)
    if output_type == 'binary':
      kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
      kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    for train_index, test_index in kf.split(X, A):
      X_train = X.loc[train_index]
      A_train = A[train_index]

      m = make_model()
      m.fit(X_train, A_train)
    
      if output_type == 'binary':
        predictions[test_index] = m.predict_proba(X.loc[test_index])[:, 1]
      elif output_type == 'continuous':
        predictions[test_index] = m.predict(X.loc[test_index])

    #assert np.isnan(predictions).sum() == 0  # Ensure no predictions are NaN
    return predictions

def mu_k_fold_fit_and_predict(make_model, X:pd.DataFrame, Y:np.array, n_splits:int, output_type:str):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    Y: array of outcomes
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"

    """
    predictions = np.full_like(Y, np.nan, dtype=float)
    if output_type == 'binary':
      kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
      kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    
    for train_index, test_index in kf.split(X, Y):
      X_train = X.loc[train_index]
      Y_train = Y.loc[train_index]
      mu = make_model()
      mu.fit(X_train, Y_train)

      if output_type =='binary':
        predictions[test_index] = mu.predict_proba(X.loc[test_index])[:, 1]
      elif output_type == 'continuous':
        predictions[test_index] = mu.predict(X.loc[test_index])

    #assert np.isnan(predictions).sum() == 0
    return predictions

### 1.3 LATE

In [7]:
"""
def late_estimator(mu1, mu0, m1, m0, p, Z, A, Y, prob = None):
  '''
  Estimator for LATE
  '''
  n = len(Y)
  phi_zy = mu1 - mu0 + Z*(Y-mu1)/p - (1-Z)*(Y-mu0)/(1-p)
  phi_za = m1 - m0 + Z*(A-m1)/p - (1-Z)*(A-m0)/(1-p)

  tau_za = phi_za.mean()
  tau_hat = phi_zy.mean()/tau_za
  phi = phi_zy - phi_za * tau_hat
  
  std_hat = math.sqrt((phi**2).mean()/tau_za**2/n)

  return tau_hat, std_hat
"""

"\ndef late_estimator(mu1, mu0, m1, m0, p, Z, A, Y, prob = None):\n  '''\n  Estimator for LATE\n  '''\n  n = len(Y)\n  phi_zy = mu1 - mu0 + Z*(Y-mu1)/p - (1-Z)*(Y-mu0)/(1-p)\n  phi_za = m1 - m0 + Z*(A-m1)/p - (1-Z)*(A-m0)/(1-p)\n\n  tau_za = phi_za.mean()\n  tau_hat = phi_zy.mean()/tau_za\n  phi = phi_zy - phi_za * tau_hat\n  \n  std_hat = math.sqrt((phi**2).mean()/tau_za**2/n)\n\n  return tau_hat, std_hat\n"

1.3.1 Partial Linear Model

In [8]:
def plm_estimator(mu_predictions, m_predictions, X, A, Y, prob=None):
    """
    Conduct PLM analysis using DML for a continuous outcome and treatment.
    
    Args:
    y: outcome variable
    A: treatment variable
    X: List of covariates
    
    Returns:
    Estimated PLM effect of the treatment on the outcome.
    """
   
    # Residuals for outcome and treatment
    Y_res = Y - (mu_predictions)  
    A_res = A - (m_predictions)

    # Convert to numpy array and reshape
    A_res_array = A_res.values.reshape(-1, 1)  
    Y_res_array = Y_res.values 

    # Combine covariates with treatment residuals
    X_with_A_res = np.hstack((X, A_res_array))

    # Estimate the treatment effect with linear regression on residuals
    plm_model = LinearRegression().fit(X_with_A_res, Y_res)

    # Estimate the treatment effect with linear regression on residuals
    #plm_estimate = LinearRegression().fit(A_res.reshape(-1, 1), Y_res)
    
    return plm_model

### 1.4 Run a trial

In [9]:
def run(df, outcome_l, treatment_l, block_l, fe, stationary_c):

    df_1 = df[outcome_l+treatment_l+block_l]
    df_1 = df_1.dropna()

    outcome = df_1[outcome_l].reset_index(drop=True).squeeze()
    treatment = df_1[treatment_l].reset_index(drop=True).squeeze()
    #instrument = df_1[instrument_l].reset_index(drop=True).squeeze()
    block = df_1[block_l].reset_index(drop=True)

    # Cross-fitting
    #p = p_k_fold_fit_and_predict(make_p_model, X=block, Z=instrument, n_splits=10)
    m_predictions= m_k_fold_fit_and_predict(make_m_model, X=block, A=treatment, n_splits=10, output_type="continuous")
    mu_predictions= mu_k_fold_fit_and_predict(make_mu_model, X=block, Y=outcome, n_splits=10, output_type="continuous")
    plm_model = plm_estimator(mu_predictions, m_predictions, X=block, A=treatment, Y=outcome, prob = None)
    tau_hat = plm_model.coef_[0].mean()  # Estimated effect
    std_hat = plm_model.coef_[0].std()  # Standard error
    tau_hat, std_hat, p = find_p(tau_hat, std_hat) # p-value
    return outcome_l[0], treatment_l[0], p, tau_hat, std_hat, fe, stationary_c
    

## 2. Analysis

### 2.1 `df_fp`

In [10]:
# read in the dataframe
df = pd.read_csv('df_fp.csv')
print(df.columns)

Index(['country', 't', 'onset2COWCS', 'milexp_pergdpSIPRI', 'decade',
       'democracy', 'logmountain', 'ethnic_fractionalization',
       'religion_fractionalization', 'language_fractionalization',
       'leg_british', 'opec', 'logpop_M_diff', 'logpopdens_diff',
       'logoutreg_diff', 'ecgrowth_demeaned', 'treat_agri', 'treat_mine',
       'treat_fuel', 'treat_metal', 'iv_transport', 'iv_agri', 'iv_mine',
       'iv_fuel', 'iv_metal'],
      dtype='object')


In [11]:
df1 = pd.read_csv('../../data/GVC_data/transportIV_file.csv')
df1 = df1.loc[:, ['country', 't', 'trans_outp_p']]

df = pd.merge(df, df1, on=['country', 't'])

df = df.drop(columns='iv_transport')
df = df.rename(columns={'trans_outp_p': 'iv_transport'})

'''
No longer using IV
# Define categorization function for IVs
def categorize_value(value, q1_3, q2_3):
    if value > q2_3:
        return 1
    elif value < q1_3:
        return 0
    else:
        return np.nan

# Columns to apply the transformation
columns = ['iv_transport']

# Iterate through the columns and apply the categorization function
for col in columns:
    q1_3 = df[col].quantile(1/3)
    q2_3 = df[col].quantile(2/3)
    
    df[col] = df[col].apply(lambda x: categorize_value(x, q1_3, q2_3))
'''


"\nNo longer using IV\n# Define categorization function for IVs\ndef categorize_value(value, q1_3, q2_3):\n    if value > q2_3:\n        return 1\n    elif value < q1_3:\n        return 0\n    else:\n        return np.nan\n\n# Columns to apply the transformation\ncolumns = ['iv_transport']\n\n# Iterate through the columns and apply the categorization function\nfor col in columns:\n    q1_3 = df[col].quantile(1/3)\n    q2_3 = df[col].quantile(2/3)\n    \n    df[col] = df[col].apply(lambda x: categorize_value(x, q1_3, q2_3))\n"

In [12]:
# in order to run random forest with categorical variable
df = label_encode_column(df, 'country')

In [13]:
res = pd.DataFrame(columns=['outcome', 'treatment', 'tau_hat', 'std_hat', 'p_val', 'fixed_effects', 'stationary_controls'])

In [14]:
df.columns

Index(['country', 't', 'onset2COWCS', 'milexp_pergdpSIPRI', 'decade',
       'democracy', 'logmountain', 'ethnic_fractionalization',
       'religion_fractionalization', 'language_fractionalization',
       'leg_british', 'opec', 'logpop_M_diff', 'logpopdens_diff',
       'logoutreg_diff', 'ecgrowth_demeaned', 'treat_agri', 'treat_mine',
       'treat_fuel', 'treat_metal', 'iv_agri', 'iv_mine', 'iv_fuel',
       'iv_metal', 'iv_transport'],
      dtype='object')

In [15]:
def run_all(df, outcome_l, treatment_l, block_fe_l, block_sta_l, block_other_l):
    '''
    For a given treatment i.e. sector.

    instrument_l: a list of instruments.
    '''
    res = pd.DataFrame(columns=['outcome', 'treatment', 'tau_hat', 'std_hat', 'p_val', 'fixed_effects', 'stationary_controls'])

    #for ins_l in instrument_l:
        #block_l = block_other_l

    for fe in [True, False]:
        block_l = block_other_l
        if fe:
            block_l += block_fe_l
        for sta in [True, False]:
            if sta:
                block_l += block_sta_l
                res_row = run(df, outcome_l, treatment_l, block_l, fe, sta)
                print(len(res_row))
                print(len(res.columns))
                res.loc[len(res)] = list(res_row)
    return res


Treatment: Fuel Sector

In [16]:
fuel_res = run_all(df, 
        outcome_l = ['milexp_pergdpSIPRI'], 
        treatment_l = ['treat_fuel'], 
        #instrument_l = ['iv_transport'], 
        block_fe_l = ['country', 't'], 
        block_sta_l = ['logmountain', 'ethnic_fractionalization', 'religion_fractionalization',
                       'language_fractionalization'], 
        block_other_l = ['democracy', 'logpopdens_diff', 
                         'ecgrowth_demeaned'])

/var/folders/kn/0r0f9sfj7g95tlp0xnjjq1500000gn/T/ipykernel_50254/667158211.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  z_value = estimate / std


7
7
7
7


/var/folders/kn/0r0f9sfj7g95tlp0xnjjq1500000gn/T/ipykernel_50254/667158211.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  z_value = estimate / std


In [17]:
fuel_res

,outcome,treatment,tau_hat,std_hat,p_val,fixed_effects,stationary_controls
0,milexp_pergdpSIPRI,treat_fuel,0.0,-127.5718,0.0,True,True
1,milexp_pergdpSIPRI,treat_fuel,0.0,-126.9249,0.0,False,True


Treatment: Agriculture

In [18]:
agri_res = run_all(df, 
        outcome_l = ['onset2COWCS'], 
        treatment_l = ['treat_agri'], 
        instrument_l = ['iv_transport'],
        block_fe_l = ['country', 't'], 
        block_sta_l = ['logmountain', 'ethnic_fractionalization', 'religion_fractionalization',
                       'language_fractionalization'], 
        block_other_l = ['democracy', 'logpopdens_diff', 
                         'ecgrowth_demeaned'])

TypeError: run_all() got an unexpected keyword argument 'instrument_l'

In [ ]:
agri_res

,outcome,treatment,instrument,tau_hat,std_hat,p_val,fixed_effects,stationary_controls
0,onset2COWCS,treat_agri,iv_transport,0.0423,0.1219,0.7285,True,True
1,onset2COWCS,treat_agri,iv_transport,0.0432,0.1230,0.7254,True,False
2,onset2COWCS,treat_agri,iv_transport,0.0413,0.1238,0.7389,False,True
3,onset2COWCS,treat_agri,iv_transport,0.0860,0.2297,0.7082,False,False
4,onset2COWCS,treat_agri,iv_agri,0.0020,0.0278,0.9423,True,True
5,onset2COWCS,treat_agri,iv_agri,0.0067,0.0315,0.8310,True,False
6,onset2COWCS,treat_agri,iv_agri,0.0012,0.0299,0.9673,False,True
7,onset2COWCS,treat_agri,iv_agri,0.0049,0.0320,0.8772,False,False


Treatment: Metal

In [ ]:
metal_res = run_all(df, 
        outcome_l = ['onset2COWCS'], 
        treatment_l = ['treat_metal'], 
        instrument_l = [['iv_transport'], ['iv_metal']], 
        block_fe_l = ['country', 't'], 
        block_sta_l = ['logmountain', 'ethnic_fractionalization', 'religion_fractionalization',
                       'language_fractionalization'], 
        block_other_l = ['democracy', 'logpopdens_diff', 
                         'ecgrowth_demeaned'])

In [ ]:
metal_res

,outcome,treatment,instrument,tau_hat,std_hat,p_val,fixed_effects,stationary_controls
0,onset2COWCS,treat_metal,iv_transport,-0.1283,1.6247,0.9371,True,True
1,onset2COWCS,treat_metal,iv_transport,-0.0416,0.8232,0.9597,True,False
2,onset2COWCS,treat_metal,iv_transport,0.1233,0.8981,0.8908,False,True
3,onset2COWCS,treat_metal,iv_transport,-0.1377,0.4960,0.7813,False,False
4,onset2COWCS,treat_metal,iv_metal,-0.0779,0.0474,0.1002,True,True
5,onset2COWCS,treat_metal,iv_metal,-0.0871,0.0482,0.0709,True,False
6,onset2COWCS,treat_metal,iv_metal,-0.0859,0.0496,0.0837,False,True
7,onset2COWCS,treat_metal,iv_metal,-0.0785,0.0500,0.1166,False,False


Treatment: Mining

In [ ]:
mine_res = run_all(df, 
        outcome_l = ['onset2COWCS'], 
        treatment_l = ['treat_mine'], 
        instrument_l = [['iv_transport'], ['iv_mine']], 
        block_fe_l = ['country', 't'], 
        block_sta_l = ['logmountain', 'ethnic_fractionalization', 'religion_fractionalization',
                       'language_fractionalization'], 
        block_other_l = ['democracy', 'logpopdens_diff', 
                         'ecgrowth_demeaned'])

In [ ]:
mine_res

,outcome,treatment,instrument,tau_hat,std_hat,p_val,fixed_effects,stationary_controls
0,onset2COWCS,treat_mine,iv_transport,0.1547,0.6230,0.8039,True,True
1,onset2COWCS,treat_mine,iv_transport,0.1485,0.9678,0.8780,True,False
2,onset2COWCS,treat_mine,iv_transport,0.0097,0.6017,0.9872,False,True
3,onset2COWCS,treat_mine,iv_transport,0.0387,0.2257,0.8638,False,False
4,onset2COWCS,treat_mine,iv_mine,-0.0563,0.0814,0.4896,True,True
5,onset2COWCS,treat_mine,iv_mine,-0.0665,0.0908,0.4641,True,False
6,onset2COWCS,treat_mine,iv_mine,-0.0656,0.0866,0.4492,False,True
7,onset2COWCS,treat_mine,iv_mine,-0.0674,0.0897,0.4527,False,False


Final Result:

In [19]:
dfs = [fuel_res] #keep fuel sector only, agri_res, metal_res, mine_res]
stacked_df = pd.concat(dfs)
final_res = stacked_df.reset_index(drop=True)

In [20]:
final_res.insert(0, 'gvc_type', 'forward')
final_res

,gvc_type,outcome,treatment,tau_hat,std_hat,p_val,fixed_effects,stationary_controls
0,forward,milexp_pergdpSIPRI,treat_fuel,0.0,-127.5718,0.0,True,True
1,forward,milexp_pergdpSIPRI,treat_fuel,0.0,-126.9249,0.0,False,True


In [21]:
final_res.to_csv('forward_res.csv', index=False)

## Appendix:

For estimating the local average treatment effect under the monotone instrument assumption, there is a double-machine learning approach that works with generic supervised learning approaches. Here, we want an estimator $\hat{\tau}^{\mathrm{LATE}}$ for the parameter
$$
\tau^{\mathrm{LATE}}=\frac{\mathbb{E}[\mathbb{E}[Y \mid X, Z=1]-\mathbb{E}[Y \mid X, Z=0]]}{\mathbb{E}[\mathrm{P}(A=1 \mid X, Z=1)-\mathrm{P}(A=1 \mid X, Z=0)]}
$$
To define the estimator, it's convenient to introduce some additional notation. First, we define the nuisance functions:
$$
\begin{aligned}
\mu(z, x) & =\mathbb{E}[Y \mid z, x] \\
m(z, x) & =\mathrm{P}(A=1 \mid x, z) \\
p(x) & =\mathrm{P}(Z=1 \mid x) .
\end{aligned}
$$
We also define the score $\phi$ by:
$$
\begin{aligned}
& \phi_{Z \rightarrow Y}(\mathbf{X} ; \mu, p) \triangleq \mu(1, X)-\mu(0, X)+\frac{Z(Y-\mu(1, X))}{p(X)}-\frac{(1-Z)(Y-\mu(0, X))}{1-p(X)} \\
& \phi_{Z \rightarrow A}(\mathbf{X} ; m, p) \triangleq m(1, X)-m(0, X)+\frac{Z(A-m(1, X))}{p(X)}-\frac{(1-Z)(A-m(0, X))}{1-p(X)} \\
& \phi(\mathbf{X} ; \mu, m, p, \tau) \triangleq \phi_{Z \rightarrow Y}(\mathbf{X} ; \mu, p)-\phi_{Z \rightarrow A}(\mathbf{X} ; m, p) \times \tau
\end{aligned}
$$
Then, the estimator is defined by a two stage procedure:
1. Fit models $\hat{\mu}, \hat{m}, \hat{p}$ for each of $\mu, m, p$ (using supervised machine learning).
2. Define $\hat{\tau}^{\mathrm{LATE}}$ as the solution to $\frac{1}{n} \sum_i \phi\left(\mathbf{X}_i ; \hat{\mu}, \hat{m}, \hat{p}, \hat{\tau}^{\mathrm{LATE}}\right)=0$. That is,
$$
\hat{\tau}^{\mathrm{LATE}}=\frac{\frac{1}{n} \sum_i \phi_{Z \rightarrow Y}\left(\mathbf{X}_i ; \hat{\mu}, \hat{p}\right)}{\frac{1}{n} \sum_i \phi_{Z \rightarrow A}\left(\mathbf{X}_i ; \hat{m}, \hat{p}\right)}
$$
It may help intuitions to notice that the double machine learning estimator of the LATE is effectively the double machine learning estimator of of the average treatment effect of $Z$ on $Y$ divided by the double machine learning estimator of the average treatment effect of $Z$ on $A$.
The nuisance functions can be estimated by:
1. fit a model $\hat{\mu}$ that predicts $Y$ from $Z, X$ by minimizing mean square error
2. fit a model $\hat{m}$ that predicts $A$ from $Z, X$ by minimizing mean cross-entropy
3. fit a model $\hat{p}$ that predicts $Z$ from $X$ by minimizing mean cross-entropy.